# HELE coal calculations

### Introduction

>*Research commissioned by the Turnbull government has ­estimated Australia’s emissions would be cut by up to 27 per cent if the country’s coal-based power generation ran on "ultra-super-critical technology", used in countries around the world.*

[Matt Canavan backs technology to cut our carbon emissions](
http://www.theaustralian.com.au/business/mining-energy/matt-canavan-backs-technology-to-cut-our-carbon-emissions/news-story/54c389f4553adbaf42c9f043f7518f0a), Tuesday 17th January 2017, *The Australian*

Josh Frydenberg was unable to quantify the cost of this technology when questioned on [Radio National (24th Jan 2017)](http://www.abc.net.au/radionational/programs/breakfast/no-plans-to-change-ret-frydenberg/8206540). This analysis attempts to firstly replicate the results prepared by the Department of Industry, Innovation and Science and reported in the media. In addition to this, the costs of achieving this reduction using "ultra-super-critical technology" is also quantified. This is compared with other options.

### Summary
* Emissions from the electricity sector could be reduced by **27%** if the exiting coal power stations were replaced with ultra supercritical coal stations.
* Approximately **20 GW** of new ultra supercritical coal power stations would be required.
* The 20 GW required would cost approximately **\$62 billion**.
* The same emissions reduction could be achieved with **13-19 GW** of renewable energy, costing **\$24-34** billion
* \$62 billion invested in renewable capacity could reduce emission in the electricity sector by **50% to 60%**


### Approach

This analysis quantifies the hypothetical emissions reduction achievable by replacing the current fleet of subcritical and supercritical power stations with new ultra super-critical technology. Generation data from individual power stations in 2016, combined with the latest emissions factor assumptions [from AEMO](https://www.aemo.com.au/Electricity/National-Electricity-Market-NEM/Planning-and-forecasting/National-Transmission-Network-Development-Plan/NTNDP-database) was used to calculate energy output and the resulting emissions. The energy was assumed to be replaced with ultra super-critical technology, with an emissions intensity of 700g/kWh (0.7 tonne/MWh). From this, the hypothetical reduction of emissions can be quantified.

The energy output from the existing coal fired powerstation can also be used to estimate the quantify for the capacity of new ultra super-critical technology. To determine this, a capacity factor of 0.85 was used. This capacity can be combined with capital cost estimates from the [Australian Power Generation Technology Report](http://www.co2crc.com.au/wp-content/uploads/2016/04/LCOE_Report_final_web.pdf) to estimate total costs.

The costs and emissions outcome were compared with renewable energy using a similar approach. In the first case, the emissions reductions were assumed to be achieved, albeit using renewable energy. In the second case, the same amount of money was assumed to be spent on renewable energy capacity, and the emissions reductions calculated. This was based on the high end of the cost recently [reported in the AFR](http://www.afr.com/business/energy/solar-energy/solar-closing-cost-gap-with-wind-conventional-power-20170113-gtqw91) at between \$1,570-\$1,780 / kW.

### Data and code

The data (emissions intensity, station energy output) and code to arrive at these results are available on [github](https://github.com/dylanjmcconnell/hele_coal) (and below), should anyone wish to examine or replicate the analyis. The raw genaration data is in units of GWh (as generated) and the raw emissions data is in units of tonnes CO₂-eq (as generated).

In [1]:
import pandas as pd

#data
df_stations = pd.read_csv('station_emissions.csv')

#Assumptions:
ei = 0.7 #HELE coal plant has emission intensity of 700 g/kWh (0.7 tonnes/MWh)
cf_coal = 0.85 #capacity factor new coal
cost_coal = 3100 #Cost of new installed capacity (from APGTR, $3,100, USC black coal)
cost_re = 1780 # Cost of new installed capacity (from AFR)
cf_re = 0.3 #capacity factor new renewable

In [2]:
df_fuel = df_stations.groupby('CATEGORY').sum()[['GWH','EMISSIONS']]
df_fuel.EMISSIONS = df_fuel.EMISSIONS/1000000 #convert to MT
df_fuel

,GWH,EMISSIONS
CATEGORY,,
Biomass,577.650409,NaN
Black Coal,104078.102940,90.996559
Brown Coal,47021.910707,58.104190
Distillate,133.722916,0.064450
Gas,23949.254238,11.314360
Hydro,25484.696914,NaN
Solar,481.955180,NaN
Wind,12382.680455,NaN


In [3]:
df_coal = df_fuel.ix[['Brown Coal','Black Coal']]
df_coal

,GWH,EMISSIONS
CATEGORY,,
Brown Coal,47021.910707,58.104190
Black Coal,104078.102940,90.996559


In [4]:
print df_coal.sum()

GWH          151100.013647
EMISSIONS       149.100749
dtype: float64


In [5]:
#Emissions caculation
total_emissions_2016 = df_fuel.sum().EMISSIONS
coal_emissions_2016 = df_coal.sum().EMISSIONS
coal_output_2016 = df_coal.sum().GWH

#Assume new HELE coal plant has emission intensity of 700 g/kWh (0.7 tonnes/MWh)
hele_emissions = ei * coal_output_2016 /1000
emissions_decrease = coal_emissions_2016 - hele_emissions


print "Absolute decrease emissions: {0:.2f} MT".format(emissions_decrease)
print "Percentage (electricity sector) decrease: {0:.2f}% ".format(emissions_decrease/total_emissions_2016*100)
print "Percentage (coal sector only) decrease: {0:.2f}% ".format(emissions_decrease/coal_emissions_2016*100)

Absolute decrease emissions: 43.33 MT
Percentage (electricity sector) decrease: 27.00% 
Percentage (coal sector only) decrease: 29.06% 


In [6]:
#Capacity and Cost calculation

#Capacity factor of new coal plant
new_capacity = coal_output_2016 / (cf_coal * 8760)
print "HELE Capacity require: {0:.2f} GW".format(new_capacity)

#Cost of new installed capacity (from Australian Power Generation Technology Report, $3,100, USC black coal)
total_capex = new_capacity * cost_coal
print "Total cost: ${0:.2f} billion".format(total_capex/1000)

HELE Capacity require: 20.29 GW
Total cost: $62.91 billion


In [7]:
#Comparison with RE
coal_emissions_intensity = df_coal.EMISSIONS/df_coal.GWH * 1000
print coal_emissions_intensity

CATEGORY
Brown Coal    1.235683
Black Coal    0.874310
dtype: float64


In [8]:
#Replacing 100% black coal, RE capacity factor = 30%
re_cf = 0.30
re_energy = emissions_decrease/(coal_emissions_intensity['Black Coal'])
re_capacity = re_energy/(re_cf*8.760)
print re_capacity
print "RE capacity: {0:.2f} GW".format(re_capacity)

#Cost
total_capex_re = re_capacity * cost_re
print "Total cost: ${0:.2f} billion".format(total_capex_re/1000)

18.8584128687
RE capacity: 18.86 GW
Total cost: $33.57 billion


In [9]:
#Replacing 100% brown coal, RE capacity factor = 30%
re_energy = emissions_decrease/(coal_emissions_intensity['Brown Coal'])

re_capacity = re_energy/(re_cf*8.760)
print "RE capacity: {0:.2f} GW".format(re_capacity)

#Cost
total_capex_re = re_capacity * cost_re
print "Total cost: ${0:.2f} billion".format(total_capex_re/1000)

RE capacity: 13.34 GW
Total cost: $23.75 billion


In [14]:
#Spending $62 billion on Renewable Capacity
new_re_capacity = 62./cost_re * 1000
print "New RE capacity: {0:.2f} GW".format(new_re_capacity)

#Emissions reduction, replacing black coal, RE capacity factor = 30%
re_energy = new_re_capacity * 8.760 * re_cf
black_emissions_reductions = re_energy * coal_emissions_intensity['Black Coal']

print "Absolute decrease emissions: {0:.2f} MT".format(black_emissions_reductions)
print "Percentage (electricity sector) decrease: {0:.2f}% ".format(black_emissions_reductions/total_emissions_2016*100)

print black_emissions_reductions / coal_emissions_2016

New RE capacity: 34.83 GW
Absolute decrease emissions: 80.03 MT
Percentage (electricity sector) decrease: 49.87% 
0.536763313991


In [11]:
#Emissions reduction, replacing brown coal first, then black coal, RE capacity factor = 30%
brown_emissions_reductions = df_coal['EMISSIONS'].ix['Brown Coal']

#(Black coal reduction is assumed to be )
black_energy_displaced = re_energy - df_coal['GWH'].ix['Brown Coal']/1000
black_emissions_reductions = black_energy_displaced * coal_emissions_intensity['Black Coal']

total_emissions_reductions = black_emissions_reductions + brown_emissions_reductions

print "Absolute decrease emissions: {0:.2f} MT".format(total_emissions_reductions)
print "Percentage (electricity sector) decrease: {0:.2f}% ".format(total_emissions_reductions/total_emissions_2016*100)



Absolute decrease emissions: 97.02 MT
Percentage (electricity sector) decrease: 60.46% 
